# Segmenting and Clustering Neighborhoods in Toronto
---

## Scrapping Toronto Neighborhoods Data from Wikipedia

*This section scraps Toronto Neighborhoods Data from wikipedia. The Scrapped data is cleaned and converted to pandas dataframe*

In [1]:
#import libraries
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable'})

In [3]:
ths = My_table.findAll('th')
headings = [th.text.strip() for th in ths] # Create a list of column headers using the ths marker in the html

table_rows = My_table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip().replace('/',',') for tr in td if tr.text.strip()] # This is separating Neighborhoods with the same postal code with ","
    if row:
        l.append(row)
        
df = pd.DataFrame(l, columns = headings)

In [4]:
# Dropping rows with 'Not assigned' Borough
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True) 
df.reset_index(drop=True)

#Replace missing Neighborhood values with their respective Borough
df['Neighborhood'].fillna(df['Borough'], inplace=True)
df

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [5]:
df.shape

(103, 3)

## Adding Latitude and Longitude Coordinates to Neighborhoods in the Data from Wikipedia

*In this section, I translate neighborhoods into latitudes and longitude make the data usable in Foursquare using the pgeocode package. The geocoder package didn't work for me and for the purpose of a project I will be working on, I needed to figure a way to automatically generate geo attibutes so I decided not to use the csv file provided*

In [6]:
import pgeocode

In [7]:
list_lat = []
list_long = []

for index,row in df.iterrows():
    Postal_code = row['Postal code']
    nomi =  pgeocode.Nominatim('ca')
    results = nomi.query_postal_code(Postal_code)
    lat = results['latitude']
    long = results['longitude']
    
    list_lat.append(lat)
    list_long.append(long)
    
df['Latitude'] = list_lat   

df['Longitude'] = list_long

df

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6518,-79.5076
165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
168,M7Y,East Toronto,Business reply mail Processing CentrE,43.7804,-79.2505
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.6325,-79.4939


## Explore and Cluster Neighboods

*In this section I explore the various neighborhoods in Toronto and replicate the New York City analysis using boroughs containing "Toronto".*

In [32]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

#### Use geopy library to get the lat and long of Toronto

In [33]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
#Drop row with missing long and lat values
df=df.dropna(subset=['Longitude'])
df=df.dropna(subset=['Latitude'])

#### Create a map of Toronto with neighborhoods superimposed

In [35]:
# create map of Toronto using latitude and longitude values for all Boroughs
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [36]:
# Selecting data with only boroughs that contains "Toronto"
Toronto_data = df[df['Borough'].str.contains("Toronto", na=False)]
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
22,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
30,M4E,East Toronto,The Beaches,43.6784,-79.2941


In [37]:
# create map of Toronto using latitude and longitude values for only Boroughs that contains the word "Toronto"
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '*****' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Generate venues close to each neighbourhood

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,limit=100):
    """This function generates a maximum of 100 venues within 500 radius of each neighbour.
    It uses the Foursquare API.
    """
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst
 Quay , South Niagara , Island airport
Rosed

In [40]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1603, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park , Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park , Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park , Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Analyze Each Neighborhood

In [41]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
print(Toronto_onehot.shape)
Toronto_onehot.head()

(1603, 216)


,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Group ros by neighborhood and take the mean frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped.head()

(39, 216)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.0,0.011364,0.022727,0.000000,0.000000,0.0,0.011364,...,0.000000,0.0,0.0,0.011364,0.000000,0.000000,0.0,0.000000,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.025641,0.0,0.000000,0.025641,0.025641,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2,Business reply mail Processing CentrE,0.066667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.016393,0.000000,0.0,0.000000,0.000000,0.000000,0.016393,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.016393
4,Central Bay Street,0.000000,0.000000,0.0,0.011364,0.000000,0.000000,0.000000,0.0,0.000000,...,0.011364,0.0,0.0,0.000000,0.011364,0.011364,0.0,0.011364,0.0,0.000000


In [43]:
#Top 5 most common venues in each neighborhood
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.06
2                Café  0.06
3  Seafood Restaurant  0.05
4          Restaurant  0.05


----Brockton , Parkdale Village , Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business reply mail Processing CentrE----
                venue  freq
0          Restaurant  0.20
1         Coffee Shop  0.13
2         Yoga Studio  0.07
3  Italian Restaurant  0.07
4   Martial Arts Dojo  0.07


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst
 Quay , South Niagara , Island airport----
                  venue  freq
0           Coffee Shop  0.08
1            Restaurant  0.07
2  Gym / Fitness Center  0.05
3    Italian Restaurant  0.05
4                  Café  0.05


----Central Bay Street----
              

In [44]:
def return_most_common_venues(row, num_top_venues):
    '''This function sorts venues in descending order'''
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create dataframe with the top 10 venues in each neighborhood

In [45]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Japanese Restaurant,Bakery,Beer Bar,Cocktail Bar,Farmers Market
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Chiropractor,Brewery,French Restaurant,Sandwich Place,Japanese Restaurant
2,Business reply mail Processing CentrE,Restaurant,Coffee Shop,Yoga Studio,Bank,Martial Arts Dojo,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Italian Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Coffee Shop,Restaurant,Bar,Gym / Fitness Center,Café,Italian Restaurant,Bank,Park,Bakery,French Restaurant
4,Central Bay Street,Coffee Shop,Clothing Store,Italian Restaurant,Japanese Restaurant,Spa,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Tea Room,Thai Restaurant


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [46]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Create dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626,0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Mexican Restaurant,Electronics Store,Thai Restaurant,Bakery
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889,0,Italian Restaurant,Gym,Martial Arts Dojo,Boutique,Bubble Tea Shop,Burrito Place,Restaurant,Ramen Restaurant,Electronics Store,Park
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Japanese Restaurant,Cosmetics Shop,Café,Middle Eastern Restaurant,Bookstore,Restaurant,Movie Theater,Ramen Restaurant
22,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Coffee Shop,Café,Restaurant,Bakery,Seafood Restaurant,Gastropub,Cosmetics Shop,Beer Bar,Diner,Italian Restaurant
30,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Health Food Store,Coffee Shop,Pub,Bakery,Trail,Gastropub,Falafel Restaurant,Event Space,Ethiopian Restaurant,Doner Restaurant


Visualize resulting clusters

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [26]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Distribution Center,Event Space,Pub,Food Truck,Spa,Italian Restaurant
6,Downtown Toronto,0,Gym,Italian Restaurant,Wings Joint,Mexican Restaurant,Bubble Tea Shop,Burger Joint,Music Venue,Café,Ramen Restaurant,Ethiopian Restaurant
13,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant,Bubble Tea Shop
22,Downtown Toronto,0,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,Bakery,Restaurant,Cosmetics Shop,Beer Bar,Diner,Breakfast Spot
30,East Toronto,0,Gastropub,Health Food Store,Trail,Bakery,Pub,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
31,Downtown Toronto,0,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Cocktail Bar,Beer Bar,Japanese Restaurant,Bakery,Sporting Goods Shop
40,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Breakfast Spot,Tea Room,Fast Food Restaurant,Spa,Gym / Fitness Center
41,Downtown Toronto,0,Grocery Store,Café,Coffee Shop,Park,Candy Store,Athletics & Sports,Baby Store,Playground,Electronics Store,Donut Shop
49,Downtown Toronto,0,Restaurant,Coffee Shop,Café,American Restaurant,Gym,Asian Restaurant,Gastropub,Hotel,Steakhouse,Thai Restaurant
50,West Toronto,0,Bakery,Furniture / Home Store,Park,Fast Food Restaurant,Art Gallery,Middle Eastern Restaurant,Gym / Fitness Center,Pizza Place,Grocery Store,Pet Store


Cluster 1 has most common venues near Cluster 1 are mainly food related

#### Cluster 2

In [27]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Central Toronto,1,Health & Beauty Service,Doner Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


Cluster 2 has a diverse category of venues with Health and beauty being the most common

#### Cluster 3

In [28]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
104,West Toronto,2,Park,Residential Building (Apartment / Condo),Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


#### Cluster 4

In [29]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downtown Toronto,3,Park,Harbor / Marina,Café,Music Venue,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
93,Central Toronto,3,Park,Lawyer,Photography Studio,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
103,Central Toronto,3,Park,Bus Line,Home Service,Trail,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


The most common venue to Cluster 4 is in the 'Park' category

#### Cluster 5

In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
111,Central Toronto,4,Gym Pool,Playground,Garden,Park,Cuban Restaurant,Cupcake Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
129,Central Toronto,4,Park,Gym,Grocery Store,Playground,Thai Restaurant,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
147,Downtown Toronto,4,Park,Grocery Store,Candy Store,Playground,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
